In [74]:
from chemwalker.gnps import Proteosafe
from chemwalker.utils import run_metfrag
from chemwalker.rwalker import *
import pandas as pd
import numpy as np
from rdkit import Chem

In [2]:
taskid = 'be5004d4dd394fe8b883633fd75ea732'

gnps_result = Proteosafe(taskid, 'V2')
gnps_result.get_gnps()
net = gnps_result.net
tabgnps = gnps_result.gnps
spectra = gnps_result.spectra

In [3]:
net.head()

,CLUSTERID1,CLUSTERID2,DeltaMZ,MEH,Cosine,OtherScore,ComponentIndex,EdgeAnnotation
0,549,574,0.000,549.0,0.9488,549.0,1,
1,10,574,-15.995,10.0,0.8224,10.0,1,
2,10,549,-15.995,10.0,0.7616,10.0,1,
3,2,390,-84.147,2.0,0.8661,2.0,3,
4,1,2,-42.967,1.0,0.9744,1.0,3,


In [4]:
tabgnps.head()

,AllGroups,DefaultGroups,EvenOdd,G1,G2,G3,G4,G5,G6,GNPSLinkout_Cluster,...,SpectrumID,UniqueFileSources,UniqueFileSourcesCount,cluster index,componentindex,number of spectra,parent mass,precursor charge,precursor mass,sum(precursor intensity)
0,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153831,gnps_net_filtered_net1.mgf,1,1,3,1,319.086,1,319.086,0
1,,G1,0,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153832,gnps_net_filtered_net1.mgf,1,2,3,1,362.053,1,362.053,0
2,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153833,gnps_net_filtered_net1.mgf,1,3,3,1,311.027,1,311.027,0
3,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,gnps_net_filtered_net1.mgf,1,4,55,1,393.135,1,393.135,0
4,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153946,gnps_net_filtered_net1.mgf,1,5,94,1,310.216,1,310.216,0


In [5]:
tabgnps.shape

(610, 28)

In [6]:
len(spectra)

610

In [7]:
spectra[0]

{'params': {'pepmass': (319.08591, None),
  'charge': [1],
  'mslevel': '2',
  'filename': 'spec-00000.mgf',
  'instrument': 'ion trap',
  'title': 'Scan Number: 1',
  'scans': '1'},
 'm/z array': array([ 94., 141., 153., 180., 181., 185.]),
 'intensity array': array([  2.,   3., 111.,   3., 999.,   1.]),
 'charge array': masked_array(data=[--, --, --, --, --, --],
              mask=[ True,  True,  True,  True,  True,  True],
        fill_value=0,
             dtype=int64)}

In [9]:
db = pd.read_csv('validation_db.psv', sep='|')
db.head()

,MonoisotopicMass,InChI,SMILES,Identifier,InChIKey2,InChIKey1,MolecularFormula,kingdom_name,superclass_name,class_name,subclass_name
0,194.05791,InChI=1/C10H10O4/c1-14-9-6-7(2-4-8(9)11)3-5-10...,COC1=C(C=CC(=C1)C=CC(=O)O)O,CCMSLIB00000220714,UHFFFAOYSA,KSEBMYQBYZTDHS,C10H10O4,Organic compounds,Phenylpropanoids and polyketides,Cinnamic acids and derivatives,Hydroxycinnamic acids and derivatives
1,176.09496,InChI=1/C10H12N2O/c11-4-3-7-6-12-10-2-1-8(13)5...,C1=CC2=C(C=C1O)C(=CN2)CCN,CCMSLIB00000220735,UHFFFAOYSA,QZAYGJVTTNCVMB,C10H12N2O,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Tryptamines and derivatives
2,176.04734,InChI=1/C10H8O3/c1-6-4-10(12)13-9-5-7(11)2-3-8...,CC1=CC(=O)OC2=C1C=CC(=C2)O,CCMSLIB00000221773,UHFFFAOYSA,HSHNITRMYYLLCV,C10H8O3,NaN,NaN,NaN,NaN
3,345.07864,"InChI=1/C12H17N4O4PS/c1-8-11(3-4-20-21(17,18)1...",CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,CCMSLIB00000221098,UHFFFAOYSA,HZSAJDVWZRBGIF,C12H18N4O4PS,Organic compounds,Organoheterocyclic compounds,Diazines,Pyrimidines and pyrimidine derivatives
4,342.11621,InChI=1/C12H22O11/c13-1-4-6(15)8(17)9(18)11(22...,C(C1C(C(C(C(O1)OCC2C(C(C(O2)(CO)O)O)O)O)O)O)O,CCMSLIB00000222747,UHFFFAOYSA,PVXPPJIGRGXGCY,C12H22O11,NaN,NaN,NaN,NaN


In [44]:
nset = list(set(net['ComponentIndex']))
nset[4]

6

In [42]:
net['ComponentIndex'].value_counts()

6     321
4     173
26     93
33     68
23     60
     ... 
35      1
71      1
72      1
61      1
98      1
Name: ComponentIndex, Length: 96, dtype: int64

In [45]:
snet = net[net['ComponentIndex']==nset[4]]

# Obtain the nodes in the component
nds = list(set(snet['CLUSTERID1'].tolist()+snet['CLUSTERID2'].tolist()))
print('Component with %s nodes' % len(nds))
nds[:10]

Component with 87 nodes


[84, 608, 145, 146, 147, 149, 150, 151, 152, 153]

In [52]:
np.where(tabgnps['cluster index']==608)[0][0]

607

In [51]:
spectra[607]

{'params': {'pepmass': (278.12811, None),
  'charge': [1],
  'mslevel': '2',
  'filename': 'spec-00000.mgf',
  'instrument': 'ion trap',
  'title': 'Scan Number: 608',
  'scans': '608'},
 'm/z array': array([ 58.995098,  60.055698,  70.065201,  76.021599, 112.087303,
        115.086998, 116.070999, 146.038895, 158.093094, 175.119705,
        201.070206, 202.054199, 219.080795, 244.076202, 261.102814]),
 'intensity array': array([ 77.519997, 300.899994, 957.140015, 852.049988, 240.460007,
        301.5     , 768.330017, 236.960007, 613.890015, 999.      ,
         78.519997, 264.640015, 117.980003,  75.120003, 161.440002]),
 'charge array': masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                    --],
              mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                     True,  True,  True,  True,  True,  True,  True],
        fill_value=0,
             dtype=int64)}

In [80]:
lid = []

for i in nds:
    j = np.where(tabgnps['cluster index']==i)[0][0]
    lid.append(run_metfrag(spectra[j], db, i))

In [81]:
len(lid)

87

In [83]:
tlid = pd.concat(lid)
tlid.shape

(4272, 16)

In [88]:
dfparam = pd.merge(snet[['CLUSTERID1', 'CLUSTERID2', 'Cosine']],                                 
                   tlid[['Identifier', 'InChI', 'cluster index', 'Score']],                      
                   left_on='CLUSTERID1', right_on='cluster index', how='left')                   
dfparam = pd.merge(dfparam, tlid[['Identifier', 'InChI', 'cluster index', 'Score']],             
                   left_on='CLUSTERID2', right_on='cluster index', how='left')
dfparam.head()

,CLUSTERID1,CLUSTERID2,Cosine,Identifier_x,InChI_x,cluster index_x,Score_x,Identifier_y,InChI_y,cluster index_y,Score_y
0,151,179,0.92,SN00025361,InChI=1S/C17H26FN3O5S/c1-20-6-8-21(9-7-20)16-1...,151,1.0,SN00150332,InChI=1S/C25H26O4/c1-16(2)18-10-8-17(9-11-18)1...,179,1.000000
1,151,179,0.92,SN00025361,InChI=1S/C17H26FN3O5S/c1-20-6-8-21(9-7-20)16-1...,151,1.0,SN00144362,InChI=1S/C23H23N3O3/c1-25-19-8-4-3-7-18(19)24-...,179,0.892764
2,151,179,0.92,SN00025361,InChI=1S/C17H26FN3O5S/c1-20-6-8-21(9-7-20)16-1...,151,1.0,SN00379019,InChI=1S/C18H30O9/c1-6-10(4)17(23)24-8-12-14(2...,179,0.856479
3,151,179,0.92,SN00025361,InChI=1S/C17H26FN3O5S/c1-20-6-8-21(9-7-20)16-1...,151,1.0,SN00299440,InChI=1S/C18H30O9/c1-6-8(3)17(23)26-15-13(22)1...,179,0.824101
4,151,179,0.92,SN00025361,InChI=1S/C17H26FN3O5S/c1-20-6-8-21(9-7-20)16-1...,151,1.0,SN00036712,InChI=1S/C25H26O4/c1-15(2)5-7-18-13-17(8-11-22...,179,0.816304


In [89]:
if sum(dfnull):
    dfparam= dfparam[np.invert(dfnull)]
method = 'RDKit7-linear'
meansc = True
dfparam['meansc'] = meansc                                                                       
dfparam.drop(['CLUSTERID1', 'CLUSTERID2'], inplace=True, axis=1)                                 
dfparam['cluster index_x'] = dfparam['cluster index_x'].astype(int)                              
dfparam['cluster index_y'] = dfparam['cluster index_y'].astype(int)                              
dfparam['method'] = method                                                                       
dfparam = dfparam.values.tolist()
dfparam[0]

[0.92,
 'SN00025361',
 'InChI=1S/C17H26FN3O5S/c1-20-6-8-21(9-7-20)16-14(26-15(11-22)17(16)23)10-19-27(24,25)13-4-2-12(18)3-5-13/h2-5,14-17,19,22-23H,6-11H2,1H3/p+1/t14-,15+,16+,17-/m1/s1',
 151,
 1.0,
 'SN00150332',
 'InChI=1S/C25H26O4/c1-16(2)18-10-8-17(9-11-18)14-23-24(26)21-13-12-20(15-22(21)29-23)28-25(27)19-6-4-3-5-7-19/h8-16,19H,3-7H2,1-2H3/b23-14-',
 179,
 1.0,
 True,
 'RDKit7-linear']

In [59]:
tabgnps.columns

Index(['AllGroups', 'DefaultGroups', 'EvenOdd', 'G1', 'G2', 'G3', 'G4', 'G5',
       'G6', 'GNPSLinkout_Cluster', 'GNPSLinkout_Network', 'LibraryID',
       'MQScore', 'MZErrorPPM', 'MassDiff', 'RTMean', 'RTStdErr', 'Smiles',
       'SpectrumID', 'UniqueFileSources', 'UniqueFileSourcesCount',
       'cluster index', 'componentindex', 'number of spectra', 'parent mass',
       'precursor charge', 'precursor mass', 'sum(precursor intensity)'],
      dtype='object')

In [60]:
import requests
import io

url_to_db = "http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task=%s&block=main&file=result_specnets_DB/" % taskid
dbmatch = pd.read_csv(io.StringIO(requests.get(url_to_db).text), sep='\t')

dbmatch.head()

,#Scan#,Adduct,CAS_Number,Charge,Compound_Name,Compound_Source,Data_Collector,ExactMass,FileScanUniqueID,INCHI,...,RT_Query,SharedPeaks,Smiles,SpecCharge,SpecMZ,SpectrumFile,SpectrumID,TIC_Query,UpdateWorkflowName,tags
0,1,M+H,NaN,1,"2-((4,6-Dimethyl-2-pyrimidinyl)thio)-N-(3-nitr...",Standard,NIST,318.079,spectra/specs_ms.pklbin1,InChI=1S/C14H14N4O3S/c1-9-6-10(2)16-14(15-9)22...,...,0,6,CC1=CC(=NC(=N1)SCC(=O)NC2=CC(=CC=C2)[N+](=O)[O...,1,319.086,spectra/specs_ms.pklbin,CCMSLIB00003153831,1119.00,UPDATE-SINGLE-ANNOTATED-GOLD,
1,10,M+H,NaN,1,Promethazine,Standard,NIST,284.135,spectra/specs_ms.pklbin10,InChI=1S/C17H20N2S/c1-13(18(2)3)12-19-14-8-4-6...,...,0,15,CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)N(C)C,1,285.142,spectra/specs_ms.pklbin,CCMSLIB00003161305,2553.64,UPDATE-SINGLE-ANNOTATED-GOLD,
2,100,M+H,NaN,1,N-(15Z-Tetracosenoyl)-1-.beta.-galactosylsphin...,Standard,NIST,809.674,spectra/specs_ms.pklbin100,InChI=1S/C48H91NO8/c1-3-5-7-9-11-13-15-17-18-1...,...,0,22,CCCCCCCCCCCCCC=CC(C(COC1C(C(C(C(O1)CO)O)O)O)NC...,1,810.682,spectra/specs_ms.pklbin,CCMSLIB00003154790,1463.06,UPDATE-SINGLE-ANNOTATED-GOLD,
3,101,M+H,NaN,1,.alpha.-Galactosyl-N-stearoylsphingosine,Standard,NIST,727.596,spectra/specs_ms.pklbin101,InChI=1S/C42H81NO8/c1-3-5-7-9-11-13-15-17-18-2...,...,0,21,CCCCCCCCCCCCCCCCCC(=O)NC(COC1C(C(C(C(O1)CO)O)O...,1,728.604,spectra/specs_ms.pklbin,CCMSLIB00003154791,2520.19,UPDATE-SINGLE-ANNOTATED-GOLD,
4,102,M+H,NaN,1,Gln-Gly,Standard,NIST,203.091,spectra/specs_ms.pklbin102,InChI=1S/C7H13N3O4/c8-4(1-2-5(9)11)7(14)10-3-6...,...,0,11,C(CC(=O)N)C(C(=O)NCC(=O)[O-])[NH3+],1,204.098,spectra/specs_ms.pklbin,CCMSLIB00003154823,1771.23,UPDATE-SINGLE-ANNOTATED-GOLD,


In [63]:
dbmatch.shape

(591, 38)

In [62]:
dbmatch.columns

Index(['#Scan#', 'Adduct', 'CAS_Number', 'Charge', 'Compound_Name',
       'Compound_Source', 'Data_Collector', 'ExactMass', 'FileScanUniqueID',
       'INCHI', 'INCHI_AUX', 'Instrument', 'IonMode', 'Ion_Source', 'LibMZ',
       'LibraryName', 'LibraryQualityString', 'Library_Class', 'MQScore',
       'MZErrorPPM', 'MassDiff', 'MoleculeExplorerDatasets',
       'MoleculeExplorerFiles', 'NumberHits', 'Organism', 'PI', 'Precursor_MZ',
       'Pubmed_ID', 'RT_Query', 'SharedPeaks', 'Smiles', 'SpecCharge',
       'SpecMZ', 'SpectrumFile', 'SpectrumID', 'TIC_Query',
       'UpdateWorkflowName', 'tags'],
      dtype='object')

In [65]:
dbmatch.rename(columns={'#Scan#': 'cluster index', 'Precursor_MZ': 'parent mass',
                       'RT_Query': 'RTMean', 'Compound_Name': 'LibraryID'}, inplace=True)
dbmatch.head()

,cluster index,Adduct,CAS_Number,Charge,LibraryID,Compound_Source,Data_Collector,ExactMass,FileScanUniqueID,INCHI,...,RTMean,SharedPeaks,Smiles,SpecCharge,SpecMZ,SpectrumFile,SpectrumID,TIC_Query,UpdateWorkflowName,tags
0,1,M+H,NaN,1,"2-((4,6-Dimethyl-2-pyrimidinyl)thio)-N-(3-nitr...",Standard,NIST,318.079,spectra/specs_ms.pklbin1,InChI=1S/C14H14N4O3S/c1-9-6-10(2)16-14(15-9)22...,...,0,6,CC1=CC(=NC(=N1)SCC(=O)NC2=CC(=CC=C2)[N+](=O)[O...,1,319.086,spectra/specs_ms.pklbin,CCMSLIB00003153831,1119.00,UPDATE-SINGLE-ANNOTATED-GOLD,
1,10,M+H,NaN,1,Promethazine,Standard,NIST,284.135,spectra/specs_ms.pklbin10,InChI=1S/C17H20N2S/c1-13(18(2)3)12-19-14-8-4-6...,...,0,15,CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)N(C)C,1,285.142,spectra/specs_ms.pklbin,CCMSLIB00003161305,2553.64,UPDATE-SINGLE-ANNOTATED-GOLD,
2,100,M+H,NaN,1,N-(15Z-Tetracosenoyl)-1-.beta.-galactosylsphin...,Standard,NIST,809.674,spectra/specs_ms.pklbin100,InChI=1S/C48H91NO8/c1-3-5-7-9-11-13-15-17-18-1...,...,0,22,CCCCCCCCCCCCCC=CC(C(COC1C(C(C(C(O1)CO)O)O)O)NC...,1,810.682,spectra/specs_ms.pklbin,CCMSLIB00003154790,1463.06,UPDATE-SINGLE-ANNOTATED-GOLD,
3,101,M+H,NaN,1,.alpha.-Galactosyl-N-stearoylsphingosine,Standard,NIST,727.596,spectra/specs_ms.pklbin101,InChI=1S/C42H81NO8/c1-3-5-7-9-11-13-15-17-18-2...,...,0,21,CCCCCCCCCCCCCCCCCC(=O)NC(COC1C(C(C(C(O1)CO)O)O...,1,728.604,spectra/specs_ms.pklbin,CCMSLIB00003154791,2520.19,UPDATE-SINGLE-ANNOTATED-GOLD,
4,102,M+H,NaN,1,Gln-Gly,Standard,NIST,203.091,spectra/specs_ms.pklbin102,InChI=1S/C7H13N3O4/c8-4(1-2-5(9)11)7(14)10-3-6...,...,0,11,C(CC(=O)N)C(C(=O)NCC(=O)[O-])[NH3+],1,204.098,spectra/specs_ms.pklbin,CCMSLIB00003154823,1771.23,UPDATE-SINGLE-ANNOTATED-GOLD,


In [66]:
# Subselect fields of interest
otabgnps = dbmatch.loc[dbmatch['cluster index'].isin(nds), 
                       ['cluster index', 'parent mass', 'RTMean', 
                        'LibraryID', 'Smiles', 'INCHI' ]]
otabgnps.head()

,cluster index,parent mass,RTMean,LibraryID,Smiles,INCHI
49,145,288.203,0,Ile-Arg,CCC(C)C(C(=O)NC(CCCN=C(N)N)C(=O)O)N,InChI=1S/C12H25N5O3/c1-3-7(2)9(13)10(18)17-8(1...
50,146,361.183,0,Ala-Asp-Arg,CC(C(=O)NC(CC(=O)O)C(=O)NC(CCCN=C(N)N)C(=O)O)N,InChI=1S/C13H24N6O6/c1-6(14)10(22)19-8(5-9(20)...
51,147,393.155,0,Cys-Asp-Arg,C(CC(C(=O)O)NC(=O)C(CC(=O)O)NC(=O)C(CS)N)CN=C(N)N,InChI=1S/C13H24N6O6S/c14-6(5-26)10(22)19-8(4-9...
53,149,434.229,0,Cys-Arg-Arg,C(CC(C(=O)NC(CCCN=C(N)N)C(=O)O)NC(=O)C(CS)N)CN...,InChI=1S/C15H31N9O4S/c16-8(7-29)11(25)23-9(3-1...
55,150,377.197,0,Cys-Val-Arg,CC(C)C(C(=O)NC(CCCN=C(N)N)C(=O)O)NC(=O)C(CS)N,InChI=1S/C14H28N6O4S/c1-7(2)10(20-11(21)8(15)6...


In [67]:
# have to select highest cosine score by cluster id
otabgnps.shape

(87, 6)

In [68]:
# Obtain InChIKey for structures present in GNPS
inchikey = [Chem.InchiToInchiKey(x) if type(x)==str else '' for x in otabgnps['INCHI']]

# Record the first block of InChIKey
otabgnps['InChIKey1'] = [x.split('-')[0] if type(x)==str else '' for x in inchikey]

# copy table to modify the copy
stabgnps = otabgnps.copy()

# Define fixed number o seeds to start walk, mostly for validation
seed_ctr = 0.1 # 10% of seeds in a given connected component
seed_ctr = np.ceil(sum(otabgnps['InChIKey1']!='')*seed_ctr)
seed_ctr = int(seed_ctr)

# Remove known structures for validation
exid = np.where(stabgnps['InChIKey1']!='')[0]
exid =  stabgnps.index[list(exid[seed_ctr:])]
stabgnps.loc[exid, 'InChIKey1'] = ''

In [90]:
def cand_pair(snet, tlid, method, parallel = True, meansc = True, ncors=0):
    if parallel:
        dfparam = pd.merge(snet[['CLUSTERID1', 'CLUSTERID2', 'Cosine']],
                           tlid[['Identifier', 'InChI', 'cluster index', 'Score']],                      
                           left_on='CLUSTERID1', right_on='cluster index', how='left')
        dfparam = pd.merge(dfparam, tlid[['Identifier', 'InChI', 'cluster index', 'Score']],             
                           left_on='CLUSTERID2', right_on='cluster index', how='left')                   
        dfnull = dfparam['cluster index_x'].isnull() | dfparam['cluster index_y'].isnull()               
        if sum(dfnull):
            dfparam= dfparam[np.invert(dfnull)]                                                          
        dfparam['meansc'] = meansc
        dfparam.drop(['CLUSTERID1', 'CLUSTERID2'], inplace=True, axis=1)                                 
        dfparam['cluster index_x'] = dfparam['cluster index_x'].astype(int)                              
        dfparam['cluster index_y'] = dfparam['cluster index_y'].astype(int)                              
        dfparam['method'] = method
        dfparam = dfparam.values.tolist()                                                                
        if(ncors):                                                                                       
            pool = multiprocessing.Pool(ncors)                                                           
        else: 
            pool = multiprocessing.Pool()                                                                
        candpair = pool.map(edge_weight2, dfparam)
        candpair = pd.DataFrame(candpair)                                                                
        candpair.reset_index(drop=True, inplace=True)                                                    
        # Keep track of cluster id and candidate id                                                      
        candpair[12] = candpair.apply(lambda a: '_'.join(map(str, a[[3, 1]])), axis=1 )                  
        candpair[13] = candpair.apply(lambda a: '_'.join(map(str, a[[7, 5]])), axis=1 )                  
        scandpair = candpair[[12, 13, 11]]
    return scandpair

In [75]:
def random_walk(graph, seed, restart_prob=0.8, step=0, epsilon=0.000001,
                niter=10000, sparce_matrix = True):
    p_0 = set_up_p0(seed, graph)
    thresh = 1
    p_t = np.copy(p_0)
    #sys.getsizeof(sog_not_normalized)
    # print small matrix to inspect col normalization
    # plot network with 3 nodes
    if sparce_matrix:
        w = nx.to_scipy_sparse_matrix(graph)
    else:
        w = nx.to_numpy_matrix(graph)
    w = normalize_cols(w)
    c = 0
    while (thresh > epsilon and c < niter):
        c += 1
        # first, calculate p^(t + 1) from p^(t)
        #p_t_1 = calculate_next_p(p_t, p_0)
        # p^{t+1} = (1-r)Wp^t+rp^0
        # np.dot matrix multiplication
        # Understand the role of re-start
        if sparce_matrix:
            walk = w.dot(p_t)
        else:
            walk = np.squeeze(np.asarray(np.dot(w, p_t)))
        no_restart = walk * (1 - restart_prob)
        restart = p_0 * restart_prob
        p_t_1 = np.add(no_restart, restart)
        if step:
            return p_t_1
        # calculate L1 norm of difference between p^(t + 1) and p^(t),
        # for checking the convergence condition
        thresh = np.linalg.norm(np.subtract(p_t_1, p_t), 1)
        # then, set p^(t) = p^(t + 1), and loop again if necessary
        # no deep copy necessary here, we're just renaming p
        p_t = p_t_1
    return p_t_1

In [91]:
scandpair = cand_pair(snet, tlid, method)
scandpair.head()

,12,13,11
0,151_SN00025361,179_SN00150332,0.310415
1,151_SN00025361,179_SN00144362,0.294355
2,151_SN00025361,179_SN00379019,0.287766
3,151_SN00025361,179_SN00299440,0.282215
4,151_SN00025361,179_SN00036712,0.279525


In [92]:
scandpair.shape

(981843, 3)

In [93]:
G = nx.Graph()
edge_list = scandpair.apply(lambda a: tuple(a), axis=1).tolist()
G.add_weighted_edges_from(edge_list)

In [97]:
tlid.head()

,Score,InChI,FragmenterScore_Values,MaximumTreeDepth,SmilesOfExplPeaks,MonoisotopicMass,Identifier,MolecularFormula,FormulasOfExplPeaks,InChIKey2,InChIKey1,FragmenterScore,ExplPeaks,NoExplPeaks,NumberPeaksUsed,cluster index
0,1.000000,InChI=1S/C20H39N5O8/c1-20(29)12(7-26)31-19(14(...,1256.0;708.0;1080.0;708.0;796.0;1256.0;808.0;8...,2,60.060001:[C]([C])([C])[O];70.07:[C][C]=[C][C]...,477.27986,CKC20,C20H39N5O8,60.060001:[C3H5O+2H]+H+;70.07:[C4H7N]+H+;112.0...,UHFFFAOYSA,DPRGTTRLFRFZIM,180.205031,60.060001_199.1;70.07_751.3;112.089996_256.7;1...,16,16,84
1,0.918846,InChI=1S/C26H33N7O2/c1-31(2)21-7-5-18(6-8-21)1...,653.0;765.0;610.0;348.0;753.0;753.0;961.0;653....,2,70.07:[C]1[C][C][N][C]1;112.089996:[C]1[C][C](...,477.28522,SN00031196,C26H35N7O2,70.07:[C4H7N]+H+;112.089996:[C6H10N2+H]+H+;115...,QFIPXVFZSA,WFEJVPBNJZYBGE,165.580601,70.07_751.3;112.089996_256.7;115.089996_328.8;...,13,16,84
2,0.837951,InChI=1S/C28H36N4O3/c1-20-9-11-22(12-10-20)19-...,448.0;1396.0;710.0;613.0;961.0;1672.0;958.0;40...,2,70.07:[C]1[C][C][N+][C]1;112.089996:[C]1[C][C]...,477.28657,SN00079685,C28H37N4O3+,70.07:[C4H9N-H]+;112.089996:[C6H11N2]+H+;115.0...,DQEYMECFSA,NUYXZQRZCPVUFY,151.002983,70.07_751.3;112.089996_256.7;115.089996_328.8;...,12,16,84
3,0.728663,InChI=1S/C30H39NO4/c1-19-10-9-13-24-16-20(2)21...,796.0;796.0;696.0;1713.0;1061.0;796.0;1756.0;1...,2,60.060001:[C]([C])([C])[O];70.07:[C]([C]=[C])[...,477.28791,DMO68,C30H39NO4,60.060001:[C3H6O+H]+H+;70.07:[C5H8+H]+H+;112.0...,BBXOWAOSSA,LYHWACCLUSDGME,131.308827,60.060001_199.1;70.07_751.3;112.089996_256.7;1...,13,16,84
4,0.686930,InChI=1S/C30H39NO4/c1-18-10-9-13-24-28(33)21(4...,696.0;1492.0;1713.0;1061.0;2021.0;2120.0;1144....,2,70.07:[C]([C])[C][C][C];112.089996:[C]([C])(C(...,477.28791,SN00380428,C30H39NO4,70.07:[C5H10]+;112.089996:[C7H10O+H]+H+;116.07...,ZPSJVCBQSA,JVHIPYJQMFNCEK,123.788181,70.07_751.3;112.089996_256.7;116.07_199.9;120....,12,16,84


In [99]:
glib = stabgnps.loc[stabgnps['InChIKey1']!='', 'cluster index'].index.tolist()
source = []
for i in glib:
    k,g = stabgnps.loc[i, ['InChIKey1', 'cluster index']].tolist()
    ksel = tlid[(tlid['cluster index']==g) & (tlid['InChIKey1']==k)]
    if len(ksel):
        idx = ksel['Identifier'].tolist()[0]
        print(f'InChIKey1:{k}, cluster index:{g}, Identifier:{idx}')
    else:
        print(ksel)
    source.extend([x for x in G.nodes() if bool(re.search('%s_%s' % (g,idx), x))])
source

InChIKey1:HYXQKVOADYPQEA, cluster index:145, Identifier:HMDB28901
Empty DataFrame
Columns: [Score, InChI, FragmenterScore_Values, MaximumTreeDepth, SmilesOfExplPeaks, MonoisotopicMass, Identifier, MolecularFormula, FormulasOfExplPeaks, InChIKey2, InChIKey1, FragmenterScore, ExplPeaks, NoExplPeaks, NumberPeaksUsed, cluster index]
Index: []
Empty DataFrame
Columns: [Score, InChI, FragmenterScore_Values, MaximumTreeDepth, SmilesOfExplPeaks, MonoisotopicMass, Identifier, MolecularFormula, FormulasOfExplPeaks, InChIKey2, InChIKey1, FragmenterScore, ExplPeaks, NoExplPeaks, NumberPeaksUsed, cluster index]
Index: []
Empty DataFrame
Columns: [Score, InChI, FragmenterScore_Values, MaximumTreeDepth, SmilesOfExplPeaks, MonoisotopicMass, Identifier, MolecularFormula, FormulasOfExplPeaks, InChIKey2, InChIKey1, FragmenterScore, ExplPeaks, NoExplPeaks, NumberPeaksUsed, cluster index]
Index: []
Empty DataFrame
Columns: [Score, InChI, FragmenterScore_Values, MaximumTreeDepth, SmilesOfExplPeaks, Monoisot

['145_HMDB28901']

In [102]:
len(glib)

9

In [96]:
glib

[145, 146, 147, 149, 150, 151, 152, 153, 155]

In [ ]:
def walk_conn_comp(net, spectra, tabgnps comp_index, method = 'RDKit7-linear',
                   adduct='[M+H]+', ppm=15, ispositive = True):
    snet = net[net['ComponentIndex']==comp_index]

    # Obtain the nodes in the component
    nds = list(set(snet['CLUSTERID1'].tolist()+snet['CLUSTERID2'].tolist()))
    print('Component with %s nodes' % len(nds))
    lid = []

    assert len(spectra) == len(tabgnps),"Mismatch between spectra and attributes."
    
    for i in nds:
        j = np.where(tabgnps['cluster index']==i)[0][0]
        lid.append(run_metfrag(spectra[j], db, i,
                               adduct=adduct, ppm=ppm, 
                               ispositive=ispositive))
    
    tlid = pd.concat(lid)
    scandpair = cand_pair(snet, tlid, method)
    G = nx.Graph()
    edge_list = scandpair.apply(lambda a: tuple(a), axis=1).tolist()
    G.add_weighted_edges_from(edge_list)
    glib = stabgnps.loc[stabgnps['InChIKey1']!='', 'cluster index'].tolist()
    source = []
    for g in glib:
        source.extend([x for x in G.nodes() if bool(re.search('^%s_' % g, x))])
    p_t = random_walk(G, source)

In [18]:
spectra[i]

{'params': {'pepmass': (217.16589, None),
  'charge': [1],
  'mslevel': '2',
  'filename': 'spec-00000.mgf',
  'instrument': 'ion trap',
  'title': 'Scan Number: 283',
  'scans': '283'},
 'm/z array': array([ 70.099998,  72.099998, 102.099998, 115.099998, 116.099998,
        133.100006, 158.100006, 172.100006, 199.100006, 200.100006]),
 'intensity array': array([ 95.599998,  25.57    , 613.190002, 313.48999 , 487.609985,
        347.049988, 417.380005, 826.369995, 393.309998, 999.      ]),
 'charge array': masked_array(data=[--, --, --, --, --, --, --, --, --, --],
              mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                     True,  True],
        fill_value=0,
             dtype=int64)}

In [23]:
import subprocess

precursor_ion_mode_positive = {
                             '[M+H]+': [1, 1.007276],
                             '[M+NH4]+': [18, 18.033823],
                             '[M+Na]+': [23, 22.989218],
                             '[M+K]+': [39, 38.963158],
                             '[M+CH3OH+H]+': [33, 33.033489],
                             '[M+ACN+H]+': [42, 42.033823],
                             '[M+ACN+Na]+': [64, 64.015765],
                             '[M+2ACN+H]+': [83, 83.060370],
                             '[M]+': [0, 0.0]
}
precursor_ion_mode_negative = {
                              '[M-H]-': [-1, -1.007276], 
                              '[M+Cl]-': [35, 34.969402],
                              '[M+HCOO]-': [45, 44.998201],
                              '[M+CH3COO]-': [59, 59.013851],
                              '[M]-': [0, 0.0]
}

def filter_db(db, prmass, ppm, inchifilt=True):
    if inchifilt:
        db = db[~db.InChIKey1.duplicated()]
    cnames = ['MonoisotopicMass', 'InChI', 'Identifier', 'InChIKey2', 'InChIKey1', 'MolecularFormula']
    comp = 10**6 *((db.MonoisotopicMass-prmass).abs()/db.MonoisotopicMass)
    db = db[comp <= ppm].reset_index(drop=True)
    return db[cnames]

metfrag_param = {
                # data file containing mz intensity peak pairs (one per line)
                'PeakListPath': '',
                # database parameters -> how to retrieve candidates
                'MetFragDatabaseType': 'LocalPSV',
                'LocalDatabasePath': '',
                #NeutralPrecursorMolecularFormula = C9H11Cl3NO3PS
                #DatabaseSearchRelativeMassDeviation = PPM
                'NeutralPrecursorMass': 0,
                #IonizedPrecursorMass = 349.93356
                # peak matching parameters
                'FragmentPeakMatchAbsoluteMassDeviation': 0.01,
                'FragmentPeakMatchRelativeMassDeviation': 5,
                'PrecursorIonMode': 1,
                'IsPositiveIonMode': True,
                # scoring parameters
                'MetFragScoreTypes': 'FragmenterScore',
                'MetFragScoreWeights': 1.0,
                # output
                # SDF, XLS, CSV, ExtendedXLS, ExtendedFragmentsXLS
                'MetFragCandidateWriter': 'FragmentSmilesPSV',
                'SampleName': '',
                'ResultsPath': '.',
                # following parameteres can be kept as they are
                'MaximumTreeDepth': 2,
                'MetFragPreProcessingCandidateFilter': 'UnconnectedCompoundFilter',
                'MetFragPostProcessingCandidateFilter': 'InChIKeyFilter',
                'NumberThreads': 6
}

spectrum = spectra[i] 
adduct='[M+H]+' 
ppm=15 
abs_diff=0.01
ispositive = True 
metpath='MetFrag2.3-CL.jar'

if ispositive:
    prmass = spectrum['params']['pepmass'][0]-precursor_ion_mode_positive[adduct][1]
    metfrag_param['PrecursorIonMode'] = precursor_ion_mode_positive[adduct][0]
else:
    metfrag_param['IsPositiveIonMode'] = False
    prmass = spectrum['params']['pepmass'][0]-precursor_ion_mode_negative[adduct][1]
    metfrag_param['PrecursorIonMode'] = precursor_ion_mode_negative[adduct][0]

spec = zip(spectrum['m/z array'], spectrum['intensity array'])
spec = ['%s\t%s\n' % x for x in spec]

sname = 'spec_%s_data.txt' % spectrum['params']['scans']
with open(sname, '+w') as f:
    for s in spec:
        f.write(s)

if type(db)==str:
    metfrag_param['MetFragDatabaseType'] = db
else:
    db_filt = filter_db(db, prmass, ppm)
    ndb = 'validation_db_%s.psv' % spectrum['params']['scans']
    db_filt.to_csv(ndb, sep='|', index=None)
    metfrag_param['LocalDatabasePath'] = ndb

cname = 'cand_%s' % spectrum['params']['scans']
metfrag_param['SampleName'] = cname 
metfrag_param['PeakListPath'] = sname
metfrag_param['FragmentPeakMatchRelativeMassDeviation'] = ppm
metfrag_param['FragmentPeakMatchAbsoluteMassDeviation'] = abs_diff
metfrag_param['NeutralPrecursorMass'] = prmass

pname = 'parameter_file_%s.txt' % spectrum['params']['scans']

with open(pname, '+w') as f:
    for k,v in metfrag_param.items():
        f.write(f'{k} = {v}\n')

subprocess.call(['java', '-jar', metpath, pname])
cname = 'cand_%s.psv' % spectrum['params']['scans']
metres = pd.read_csv(cname, sep='|')

os.remove(sname)
os.remove(ndb)                 
os.remove(cname)
os.remove(pname)                  

EmptyDataError: No columns to parse from file

In [79]:
def run_metfrag(spectrum, db, cluster_index, adduct='[M+H]+', ppm=15, abs_diff=0.01,
                ispositive = True, metpath='MetFrag2.3-CL.jar'):
    if ispositive:
        prmass = spectrum['params']['pepmass'][0]-precursor_ion_mode_positive[adduct][1]
        metfrag_param['PrecursorIonMode'] = precursor_ion_mode_positive[adduct][0]
    else:
        metfrag_param['IsPositiveIonMode'] = False
        prmass = spectrum['params']['pepmass'][0]-precursor_ion_mode_negative[adduct][1]
        metfrag_param['PrecursorIonMode'] = precursor_ion_mode_negative[adduct][0]

    spec = zip(spectrum['m/z array'], spectrum['intensity array'])
    spec = ['%s\t%s\n' % x for x in spec]

    sname = 'spec_%s_data.txt' % spectrum['params']['scans']
    with open(sname, '+w') as f:
        for s in spec:
            f.write(s)

    if type(db)==str:
        metfrag_param['MetFragDatabaseType'] = db
    else:
        db_filt = filter_db(db, prmass, ppm)
        ndb = 'validation_db_%s.psv' % spectrum['params']['scans']
        db_filt.to_csv(ndb, sep='|', index=None)
        metfrag_param['LocalDatabasePath'] = ndb

    cname = 'cand_%s' % spectrum['params']['scans']
    metfrag_param['SampleName'] = cname
    metfrag_param['PeakListPath'] = sname
    metfrag_param['FragmentPeakMatchRelativeMassDeviation'] = ppm
    metfrag_param['FragmentPeakMatchAbsoluteMassDeviation'] = abs_diff
    metfrag_param['NeutralPrecursorMass'] = prmass

    pname = 'parameter_file_%s.txt' % spectrum['params']['scans']

    with open(pname, '+w') as f:
        for k,v in metfrag_param.items():
            f.write(f'{k} = {v}\n')

    subprocess.call(['java', '-jar', metpath, pname])
    cname = 'cand_%s.psv' % spectrum['params']['scans']
    try:
        metres = pd.read_csv(cname, sep='|')
        metres['cluster index'] = cluster_index
    except:
        metres = pd.DataFrame()

    os.remove(sname)
    os.remove(ndb)
    os.remove(cname)
    os.remove(pname)

    return metres

In [31]:
db.shape

(694297, 11)

In [33]:
import os
edf = run_metfrag(spectra[i], db)

In [35]:
type(edf)

pandas.core.frame.DataFrame

In [36]:
tabgnps[tabgnps['parent mass']==478.289]

,AllGroups,DefaultGroups,EvenOdd,G1,G2,G3,G4,G5,G6,GNPSLinkout_Cluster,...,SpectrumID,UniqueFileSources,UniqueFileSourcesCount,cluster index,componentindex,number of spectra,parent mass,precursor charge,precursor mass,sum(precursor intensity)
83,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003156370,gnps_net_filtered_net1.mgf,1,84,6,1,478.289,1,478.289,0


In [37]:
net[(net['CLUSTERID1']==84) | (net['CLUSTERID2']==84)]

,CLUSTERID1,CLUSTERID2,DeltaMZ,MEH,Cosine,OtherScore,ComponentIndex,EdgeAnnotation
268,84,149,44.059,84.0,0.8691,84.0,6,C3H8
280,84,272,156.101,84.0,0.8936,84.0,6,C10H13N5O9P2_Adenosine 5'-diphosphate (-H2O):C...
318,84,187,-39.011,84.0,0.8304,84.0,6,
325,84,169,16.028,84.0,0.9454,84.0,6,CH4
367,84,257,41.074,84.0,0.8484,84.0,6,
374,84,256,46.021,84.0,0.9278,84.0,6,H2N2O
457,84,259,33.984,84.0,0.9693,84.0,6,
474,84,172,33.026,84.0,0.9011,84.0,6,NH3O:H3NO
497,84,185,48.000,84.0,0.8451,84.0,6,
535,84,263,-15.995,84.0,0.9855,84.0,6,


In [73]:
not len(edf)

True